In [1]:
import wntr
import pandas as pd 
import numpy as np

In [2]:
wn = wntr.network.WaterNetworkModel()
wn.add_pattern('pat1', [1])

#Adicionando o reservatório 
wn.add_reservoir('res', base_head=18.4, head_pattern='pat1', coordinates=(0,1.5))

# Adicionando os nós/conexões/junções
wn.add_junction('NP1', base_demand=0.00, demand_pattern='pat1', elevation=0,
                coordinates=(0.25,2))
wn.add_junction('NP2', base_demand=0.00, demand_pattern='pat1', elevation=0,
                coordinates=(0.25,3))

for i in range(1,61):
    wn.add_junction('N'+str(i), base_demand=0.000, 
                    demand_pattern='pat1', elevation=0.9, coordinates=(i*0.5,3))

#for j in range(1,61):
 #   wn.add_junction('N-'+str(j), base_demand=0.00009826, 
  #                  demand_pattern='pat1', elevation=0.9, coordinates=(j*0.5,4))   




    
for k in range(1,241):
    for j in range(1,61):
        wn.add_junction('N-'+str(j)+'.'+str(k), base_demand=0.0000, 
                        demand_pattern='pat1', elevation=0.9, coordinates=(j*0.5,k+3))
for k in range(1,241):
    for j in range(1,61):
        wn.add_junction('N--'+str(j)+'.'+str(k), base_demand=0.0000004094, 
                        demand_pattern='pat1', elevation=0.9, coordinates=(j*0.5-0.5,k+3))

        
        
# Adicionando as tubulações
wn.add_pipe('LR', 'res', 'NP1', length=6, diameter=0.100, 
            roughness=100, minor_loss=0.0, initial_status='OPEN')
wn.add_pipe('LP1', 'NP1', 'NP2', length=50, diameter=0.075,
            roughness=100, minor_loss=0.0, initial_status='OPEN')
wn.add_pipe('LD0', 'NP2', 'N1', length=50, diameter=0.075,
            roughness=100, minor_loss=0.0, initial_status='OPEN')

for k in range(1,29):    
    wn.add_pipe('LD'+str(k), 'N'+str(k), 'N'+str(k+1), length=1, diameter=0.075, 
                roughness=100, minor_loss=0.0, initial_status='OPEN')

for l in range(29,60):    
    wn.add_pipe('LD'+str(l), 'N'+str(l), 'N'+str(l+1), length=1, diameter=0.050, 
                roughness=100, minor_loss=0.0, initial_status='OPEN')


#for m in range(1,61):
 #   wn.add_pipe('LL'+str(m), 'N'+str(m), 'N-'+str(m), length=72, diameter=0.016, 
  #              roughness=100, minor_loss=0.0, initial_status='OPEN')


#for m in range(1,61):
 #   wn.add_pipe('LL'+str(m)+'.1', 'N-'+str(m), 'N-'+str(m)+'.1', length=0.3, diameter=0.016, 
  #              roughness=100, minor_loss=0.0, initial_status='OPEN')


                
for m in range(1,61):
    wn.add_pipe('LL'+str(m)+'.1', 'N'+str(m), 'N-'+str(m)+'.1', length=0.3, diameter=0.016, 
                roughness=100, minor_loss=0.0, initial_status='OPEN')

for k in range(2,241):
    for m in range(1,61):    
        wn.add_pipe('LL'+str(m)+'.'+str(k), 'N-'+str(m)+'.'+str(k-1), 'N-'+str(m)+'.'+str(k), length=0.3, diameter=0.016, 
                    roughness=100, minor_loss=0.0, initial_status='OPEN')

for k in range(1,241):
    for m in range(1,61):    
        wn.add_pipe('LL-'+str(m)+'.'+str(k), 'N-'+str(m)+'.'+str(k), 'N--'+str(m)+'.'+str(k), length=0.1, diameter=0.01, 
                    roughness=100, minor_loss=0.0, initial_status='OPEN')
    
    

wn.options.hydraulic.demand_model = 'PDD'
wn.options.time.hydraulic_timestep = 1
wn.options.time.report_timestep = 1
wn.options.time.duration = 2
#wn.options.hydraulic.required_pressure =  
#wn.options.hydraulic.minimum_pressure  = 0

for i in range(1,61):
    junction = wn.get_node('N'+str(i))
    junction.required_pressure = 11.4 # 20 psi = 14.065 m
    junction.minimum_pressure = 0 


#for i in range(1,61):
 #   junction = wn.get_node('N-'+str(i)+'.1')
  #  junction.required_pressure = 10 # 20 psi = 14.065 m
   # junction.minimum_pressure = 0 



for k in range(1,241):
    for i in range(1,61):
        junction = wn.get_node('N--'+str(i)+'.'+str(k))
        junction.required_pressure = 10 # 20 psi = 14.065 m
        junction.minimum_pressure = 0 


#for i in range(1,241):
#for k in range(1,61):
wn = wntr.morph.split_pipe(wn,'LP1','LP1_B','LeakLP1')
leak_node = wn.get_node('LeakLP1')
leak_node.add_leak(wn, area=0., start_time=2, end_time=3)

#wn.write_inpfile('IS_toplogy.inp', version=2.2)

sim = wntr.sim.WNTRSimulator(wn)
results = sim.run_sim()

# Plot results on the network
pressure = results.node['pressure']
display(pressure)

,NP1,NP2,N1,N2,N3,N4,N5,N6,N7,N8,...,N--52.240,N--53.240,N--54.240,N--55.240,N--56.240,N--57.240,N--58.240,N--59.240,N--60.240,res
0,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,9.643304,9.635143,9.62877,9.623979,9.620562,9.618301,9.616974,9.616347,9.616174,0.0
1,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,9.643304,9.635143,9.62877,9.623979,9.620562,9.618301,9.616974,9.616347,9.616174,0.0
2,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,9.643304,9.635143,9.62877,9.623979,9.620562,9.618301,9.616974,9.616347,9.616174,0.0


In [22]:
driponcl.to_csv('Pressure-Drip-Off-1h.csv')

## Non-leaking pressure dataset with Drip-On

In [2]:
df = pd.read_csv("Pressure-Drip-On.csv")
dripon = df.drop(['Unnamed: 0'], axis=1)
dripon

,NP1,NP2,N1,N2,N3,N4,N5,N6,N7,N8,...,N51,N52,N53,N54,N55,N56,N57,N58,N59,N60
0,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
1,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
2,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
3,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
4,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
5,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
6,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
7,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
8,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
9,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625


In [22]:
dripon1h = dripon.append([dripon]*327, ignore_index=True)
dripon1h

,NP1,NP2,N1,N2,N3,N4,N5,N6,N7,N8,...,N51,N52,N53,N54,N55,N56,N57,N58,N59,N60
0,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
1,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
2,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
3,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
4,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
3604,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
3605,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625
3606,18.330147,15.966574,12.703001,12.657177,12.612782,12.569793,12.52819,12.487953,12.44906,12.411491,...,10.786905,10.776755,10.768594,10.76222,10.75743,10.754012,10.751751,10.750424,10.749798,10.749625


## Non-leaking pressure dataset with Drip-Off

In [10]:
df = pd.read_csv("Pressure-Drip-Off-1h.csv")
dripoff = df.drop(['Unnamed: 0'], axis=1)
dripoff

,NP1,NP2,N1,N2,N3,N4,N5,N6,N7,N8,...,N51,N52,N53,N54,N55,N56,N57,N58,N59,N60
0,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
1,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
2,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
3,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
4,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
3596,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
3597,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116
3598,18.330142,15.966391,12.70264,12.656812,12.612413,12.569421,12.527815,12.487575,12.448679,12.411107,...,10.786399,10.776248,10.768086,10.761712,10.756922,10.753504,10.751243,10.749915,10.749289,10.749116


In [8]:
stds = np.array(dripon1_1)
stds

array([2.633e-06, 9.174e-05, 1.808e-04, 1.826e-04, 1.842e-04, 1.859e-04, 1.874e-04, 1.889e-04, 1.904e-04, 1.918e-04, 1.932e-04, 1.945e-04, 1.958e-04, 1.970e-04, 1.982e-04, 1.993e-04, 2.004e-04, 2.015e-04, 2.025e-04, 2.034e-04, 2.043e-04, 2.052e-04, 2.061e-04, 2.069e-04, 2.076e-04, 2.084e-04,
       2.091e-04, 2.097e-04, 2.103e-04, 2.109e-04, 2.115e-04, 2.153e-04, 2.188e-04, 2.222e-04, 2.253e-04, 2.282e-04, 2.309e-04, 2.335e-04, 2.358e-04, 2.380e-04, 2.400e-04, 2.418e-04, 2.435e-04, 2.451e-04, 2.464e-04, 2.477e-04, 2.488e-04, 2.498e-04, 2.506e-04, 2.514e-04, 2.520e-04, 2.526e-04,
       2.531e-04, 2.535e-04, 2.538e-04, 2.540e-04, 2.542e-04, 2.543e-04, 2.544e-04, 2.544e-04, 2.545e-04, 2.545e-04])

## Noise addition to simulate water pressure fluctuations

In [13]:
noises = np.zeros( (3600, 62) )

for j in range(3600):
    for i in range(62):
        noises[j,i] = np.random.normal(0,stds[i])
        
print(noises)

# 0 is the mean of the normal distribution you are choosing from
# 1 is the standard deviation of the normal distribution
# 100 is the number of elements you get in array noise

[[ 1.379e-06 -1.096e-04 -1.208e-04 ... -5.543e-05 -3.008e-04 -3.031e-04]
 [ 3.700e-06 -4.164e-06 -3.716e-05 ... -2.664e-04 -7.902e-05 -7.206e-06]
 [ 3.503e-07  1.458e-05 -2.532e-04 ...  1.448e-04 -2.392e-04 -2.775e-04]
 ...
 [-2.545e-06 -2.065e-05 -1.240e-04 ...  5.914e-04  3.281e-04  1.364e-04]
 [-3.440e-06  1.074e-04 -1.296e-04 ... -1.186e-04 -2.606e-05  1.764e-04]
 [ 1.838e-06  5.790e-05 -1.941e-04 ... -5.420e-05  9.600e-06  3.054e-04]]


In [26]:
train_data = dripon1h+noises
train_data

,NP1,NP2,N1,N2,N3,N4,N5,N6,N7,N8,...,N51,N52,N53,N54,N55,N56,N57,N58,N59,N60
8,18.330149,15.966465,12.702881,12.656946,12.612865,12.569789,12.528115,12.487954,12.449073,12.411547,...,10.786909,10.776860,10.768063,10.762099,10.757514,10.753986,10.751613,10.750369,10.749497,10.749321
9,18.330151,15.966570,12.702964,12.657327,12.612539,12.569852,12.528177,12.488066,12.449045,12.411585,...,10.786889,10.776681,10.769133,10.762340,10.756989,10.753905,10.751706,10.750158,10.749719,10.749617
10,18.330148,15.966589,12.702748,12.656967,12.612901,12.569599,12.528190,12.487892,12.449114,12.411553,...,10.786513,10.777057,10.768982,10.762688,10.757874,10.753984,10.751590,10.750569,10.749559,10.749347
11,18.330146,15.966615,12.702798,12.657327,12.612796,12.569706,12.528272,12.488185,12.449190,12.411450,...,10.786738,10.776528,10.768722,10.762197,10.756982,10.754419,10.751832,10.750588,10.749747,10.749048
12,18.330147,15.966560,12.703277,12.656986,12.612698,12.569822,12.528189,12.487683,12.448990,12.411450,...,10.786806,10.776659,10.768481,10.762476,10.757496,10.754114,10.751735,10.750505,10.749757,10.750205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,18.330147,15.966690,12.702905,12.657101,12.612888,12.570024,12.527992,12.487885,12.449172,12.411340,...,10.786899,10.776707,10.768783,10.762155,10.757651,10.753907,10.752065,10.750911,10.749378,10.749668
3604,18.330144,15.966649,12.702925,12.657462,12.612774,12.569493,12.528330,12.487818,12.448913,12.411358,...,10.787222,10.776441,10.768562,10.762249,10.757174,10.753971,10.751619,10.750368,10.750116,10.749557
3605,18.330145,15.966554,12.702877,12.656788,12.612781,12.569565,12.528388,12.487924,12.448874,12.412077,...,10.786623,10.776383,10.768419,10.762426,10.757004,10.754429,10.752059,10.751016,10.750126,10.749761
3606,18.330144,15.966682,12.702872,12.657084,12.612668,12.569879,12.528210,12.488114,12.449126,12.411377,...,10.787077,10.776751,10.768241,10.762103,10.757196,10.754048,10.751191,10.750306,10.749772,10.749801


In [27]:
train_data.to_csv('DatasetTcc.csv')